In [ ]:
import pandas as pd
import numpy as np
import json
import random
from datetime import datetime
from deap import base, creator, tools, algorithms
import io
from azure.storage.blob import BlobServiceClient
from imblearn.over_sampling import ADASYN
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import optuna
from sklearn.metrics import r2_score, precision_score, f1_score, roc_curve
import scipy.stats as sps # stats 대신 sps 라는 별명으로 불러옵니다.
import sys
sys.path.append("/home/azureuser/cloudfiles/code")

from Users.project.src.data_container.data_container import AzureStorageAccess


# ==============================================================================
#                      <<< F1 전략 최적화 엔진 (완전 통합 버전) >>>
# ==============================================================================

# ------------------------------------------------------------------------------
# [ STEP 1: 설정 ]
# ------------------------------------------------------------------------------
# track_name = 'Australian' # <--- 분석하고 싶은 트랙 이름만 여기서 변경하세요.
# initial_compound = "D" # 시뮬레이션 시작 타이어 (필요시 변경)
# ------------------------------------------------------------------------------
# [ STEP 2: 모든 함수 정의 ]
# ------------------------------------------------------------------------------
# (이곳에 우리가 만든 모든 함수, 즉 convert_numpy_types, evaluate_strategy,
#  create_random_strategy, evaluate_hybrid_strategy, custom_mutate, custom_mate 등을 모두 정의합니다.)

def convert_numpy_types(obj):
    """
    딕셔너리, 리스트를 순회하며 NumPy 타입을 Python 기본 타입으로 변환하는 함수
    """
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    return obj

def calculate_pit_proba_threshold(y_true, y_pred_proba):
    """
    ROC 곡선을 사용해 pit_proba의 최적 임계값 계산
    - 유덴 지수를 최대화하거나 F1 점수를 최적화
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    youden_index = tpr - fpr
    optimal_idx = np.argmax(youden_index)
    optimal_threshold = thresholds[optimal_idx]
    
    # F1 점수 검증 (옵션)
    y_pred_binary = (y_pred_proba >= optimal_threshold).astype(int)
    optimal_f1 = f1_score(y_true, y_pred_binary)
    
    print(f"Optimal pit_proba threshold (Youden's Index): {optimal_threshold:.3f}")
    print(f"Corresponding F1 Score: {optimal_f1:.3f}")
    return optimal_threshold

def calculate_tyre_wear_threshold(laps_data, percentile=75):
    """
    TyreLife 분포를 분석해 tyre_wear 임계값 계산
    - 백분위수 기반, LapTime과의 상관관계 보완
    """
    # TyreLife가 존재하는 데이터 필터링
    tyre_data = laps_data.dropna(subset=['TyreLife'])
    
    # 백분위수 계산
    wear_threshold = np.percentile(tyre_data['TyreLife'], percentile)
    print(f"TyreLife {percentile}th percentile threshold: {wear_threshold:.1f}")
    
    # LapTime과의 상관관계 분석 (선택적 보완)
    corr = tyre_data['TyreLife'].corr(tyre_data['LapTime'])
    if abs(corr) > 0.5:
        # 상관계수가 높으면, LapTime이 증가하는 지점에서의 TyreLife 임계값 조정
        sorted_data = tyre_data.sort_values('TyreLife')
        lap_time_increase_idx = np.where(np.diff(sorted_data['LapTime']) > sorted_data['LapTime'].mean() * 0.1)[0]
        if len(lap_time_increase_idx) > 0:
            adjusted_threshold = sorted_data['TyreLife'].iloc[lap_time_increase_idx[0]] 
            print(f"Adjusted threshold based on LapTime increase: {adjusted_threshold:.1f}")
            return adjusted_threshold
    return wear_threshold


def extract_pit_stop_data(laps_data, track_name, total_laps):
    """
    피트스톱 데이터를 추출하고 다음 컴파운드 선택을 레이블링하는 헬퍼 함수
    """
    pit_stop_list = []
    # track_name에 해당하는 데이터만 필터링
    track_data = laps_data[laps_data['GrandPrix'].str.contains(track_name, na=False)]
    
    for (gp, driver), group in track_data.groupby(['GrandPrix', 'Driver']):
        group = group.sort_values('LapNumber')
        
        # PitInTime 또는 IsPitStop 기반 피트스톱 식별
        if 'PitInTime' in group.columns and group['PitInTime'].notna().any():
            pit_stops = group[group['PitInTime'].notna()].copy()
        elif 'IsPitStop' in group.columns and group['IsPitStop'].notna().any():
            pit_stops = group[group['IsPitStop'] == 1].copy()
        else:
            # 위 두 조건에 해당하지 않을 경우, TyreWear 기반으로 추정 (Fallback)
            pit_stops = group[group['TyreWear'] >= 28].drop_duplicates(subset=['LapNumber'], keep='first').copy()
        
        # 다음 랩의 컴파운드를 현재 랩에 할당하여 '선택된 컴파운드'로 사용
        pit_stops.loc[:, 'NextCompound'] = pit_stops['Compound'].shift(-1)
        
        for idx, row in pit_stops[pit_stops['NextCompound'].notnull()].iterrows():
            pit_stop_list.append({
                'TrackTemp': row['TrackTemp'],
                'RemainingLaps': min(total_laps - row['LapNumber'], total_laps),
                'ChosenCompound': row['NextCompound']
            })
            
    return pd.DataFrame(pit_stop_list)

def find_compound_choice_thresholds(laps_data, track_name):
    """
    주어진 랩 데이터와 트랙 이름에 대해,
    다음 타이어 컴파운드 선택을 가장 잘 예측하는 최적의 임계값 (t1, t2, t3)을 찾습니다.
    
    Parameters:
    -----------
    laps_data : pd.DataFrame
        분석할 랩 데이터 (Dry 또는 Wet으로 필터링된 데이터)
    track_name : str
        분석 대상 트랙의 이름 (예: 'Abu_Dhabi')
        
    Returns:
    --------
    dict
        최적의 임계값과 해당 임계값에서의 정확도를 담은 딕셔너리
        예: {'t1': 30.0, 't2': 15, 't3': 5, 'accuracy': 0.85}
    """
    print(f"--- {track_name} 트랙의 컴파운드 선택 임계값 계산 시작 ---")
    
    # 트랙별 총 랩 수 정의
    total_laps_dict = {'Abu_Dhabi': 58, 'Monza': 53, 'Silverstone': 52}
    total_laps = total_laps_dict.get(track_name, 58) # 기본값 58랩
    
    # 피트스톱 데이터 추출
    pit_stop_df = extract_pit_stop_data(laps_data, track_name, total_laps)
    
    if pit_stop_df.empty:
        print(f"경고: {track_name}에서 유효한 피트스톱 데이터를 추출할 수 없습니다. 기본값을 반환합니다.")
        return {"t1": 30.0, "t2": 29, "t3": 11, "accuracy": 0.0}

    # 그리드 서치로 최적 임계값 찾기
    t1_values = np.arange(15.0, 40.0, 1.0)  # TrackTemp
    t2_values = np.arange(0.3, 0.9, 0.1)   # RemainingLaps 비율 (긴 주행)
    t3_values = np.arange(0.05, 0.25, 0.05) # RemainingLaps 비율 (짧은 주행)

    best_accuracy = 0
    best_thresholds = (t1_values[0], int(t2_values[0] * total_laps), int(t3_values[0] * total_laps))

    for t1 in t1_values:
        for t2_ratio in t2_values:
            for t3_ratio in t3_values:
                if t3_ratio < t2_ratio:
                    t2_laps = int(t2_ratio * total_laps)
                    t3_laps = int(t3_ratio * total_laps)
                    
                    correct = 0
                    total = len(pit_stop_df)
                    
                    if total == 0: continue

                    # 예측 로직
                    def predict_compound(row):
                        if row['TrackTemp'] > t1 or row['RemainingLaps'] > t2_laps:
                            return "HARD"
                        elif row['RemainingLaps'] > t3_laps:
                            return "MEDIUM"
                        else:
                            return "SOFT"
                    
                    pit_stop_df['PredictedCompound'] = pit_stop_df.apply(predict_compound, axis=1)
                    correct = (pit_stop_df['PredictedCompound'] == pit_stop_df['ChosenCompound']).sum()
                    accuracy = correct / total
                    
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_thresholds = (t1, t2_laps, t3_laps)

    # 최종 결과를 딕셔너리 형태로 구성
    result = {
        "t1": best_thresholds[0],
        "t2": best_thresholds[1],
        "t3": best_thresholds[2],
        "accuracy": round(best_accuracy, 2)
    }
    
    print(f"최적 임계값: t1={result['t1']}°C, t2={result['t2']}랩, t3={result['t3']}랩 (정확도: {result['accuracy']})")
    print("--- 계산 완료 ---")
    
    return result

def calculate_pit_stop_time(laps_data, track_name):
    """
    피트스톱 시간을 계산하는 함수
    
    Parameters:
    laps_data: DataFrame - 랩 데이터 (전처리된 데이터)
    track_name: str - 트랙 이름 (예: 'Abu_Dhabi')
    
    Returns:
    float - 평균 피트스톱 시간 (초)
    """
    
    # 데이터 유효성 검사
    if laps_data is None or laps_data.empty:
        print(f"경고: 데이터가 비어있습니다. 기본값(22초) 사용.")
        return 22.0
    
    # 필요한 컬럼들이 존재하는지 확인
    required_columns = ['GrandPrix', 'PitInTime', 'PitOutTime', 'LapNumber', 'Driver']
    missing_columns = [col for col in required_columns if col not in laps_data.columns]
    
    if missing_columns:
        print(f"경고: 필요한 컬럼들이 누락되었습니다: {missing_columns}")
        print(f"사용 가능한 컬럼들: {list(laps_data.columns)}")
        return 22.0
    
    # 해당 트랙 데이터 필터링
    try:
        track_data = laps_data[laps_data['GrandPrix'].str.contains(track_name, na=False)].copy()
        if track_data.empty:
            print(f"경고: {track_name} 트랙 데이터를 찾을 수 없습니다.")
            print(f"사용 가능한 트랙들: {laps_data['GrandPrix'].unique()}")
            return 22.0
            
        print(f"{track_name} 데이터 발견: {len(track_data)} 행")
        
    except Exception as e:
        print(f"경고: 데이터 필터링 중 오류 발생: {e}. 기본값(22초) 사용.")
        return 22.0

    pit_times = []
    
    # 각 드라이버별로 피트스톱 시간 계산
    for driver in track_data['Driver'].unique():
        driver_data = track_data[track_data['Driver'] == driver].sort_values('LapNumber').reset_index(drop=True)
        
        # PitInTime이 있는 랩들 찾기
        pit_in_laps = driver_data[driver_data['PitInTime'].notna()]
        
        for _, pit_in_row in pit_in_laps.iterrows():
            try:
                pit_in_lap = pit_in_row['LapNumber']
                pit_in_time_str = str(pit_in_row['PitInTime'])
                
                # 해당 드라이버의 다음 랩들에서 PitOutTime 찾기
                next_laps = driver_data[driver_data['LapNumber'] > pit_in_lap]
                pit_out_row = next_laps[next_laps['PitOutTime'].notna()].head(1)
                
                if pit_out_row.empty:
                    print(f"경고: {driver} 드라이버의 랩 {pit_in_lap}에서 PitOutTime을 찾을 수 없습니다. 건너뜁니다.")
                    continue
                
                pit_out_time_str = str(pit_out_row['PitOutTime'].iloc[0])
                
                # 시간 파싱
                try:
                    # timedelta 형식 처리
                    if 'days' in pit_in_time_str:
                        pit_in_time = pd.to_timedelta(pit_in_time_str)
                    else:
                        pit_in_time = pd.to_timedelta(f'0 days {pit_in_time_str}')
                        
                    if 'days' in pit_out_time_str:
                        pit_out_time = pd.to_timedelta(pit_out_time_str)
                    else:
                        pit_out_time = pd.to_timedelta(f'0 days {pit_out_time_str}')
                    
                except Exception as parse_error:
                    print(f"경고: {driver} 드라이버의 시간 파싱 오류: {parse_error}. 건너뜁니다.")
                    continue
                
                # 피트스톱 시간 계산
                pit_stop_time = (pit_out_time - pit_in_time).total_seconds()
                
                # 유효성 검사 (10초 ~ 120초)
                if pit_stop_time <= 0:
                    print(f"경고: {driver} 드라이버의 랩 {pit_in_lap}에서 음수 또는 0인 피트스톱 시간: {pit_stop_time:.2f}초. 건너뜁니다.")
                    continue
                    
                if 10 < pit_stop_time < 120:
                    pit_times.append(pit_stop_time)
                    print(f"{driver} 드라이버 랩 {pit_in_lap}: 피트스톱 시간 {pit_stop_time:.2f}초")
                else:
                    print(f"경고: {driver} 드라이버의 랩 {pit_in_lap}에서 비정상적인 피트스톱 시간: {pit_stop_time:.2f}초. 건너뜁니다.")
                    
            except Exception as e:
                print(f"경고: {driver} 드라이버의 데이터 처리 중 오류: {e}. 건너뜁니다.")
                continue

    # 결과 처리
    if not pit_times:
        print(f"경고: {track_name}에 유효한 피트스톱 시간이 없습니다. 기본값(22초) 사용.")
        return 22.0

    # 평균 피트스톱 시간 계산
    avg_pit_stop_time = np.mean(pit_times)
    print(f"\n{track_name} 평균 피트스톱 시간: {avg_pit_stop_time:.2f}초 (유효 데이터 수: {len(pit_times)})")
    return avg_pit_stop_time

def analyze_pit_stops_by_track(laps_data):
    """
    모든 트랙별 피트스톱 시간 분석
    """
    print("=== 트랙별 피트스톱 시간 분석 ===")
    
    if 'GrandPrix' not in laps_data.columns:
        print("GrandPrix 컬럼이 없습니다.")
        return
    
    tracks = laps_data['GrandPrix'].unique()
    results = {}
    
    for track in tracks:
        track_name = track.split('_')[-3] if '_' in track else track  # 'Abu_Dhabi' 추출
        print(f"\n--- {track} 분석 ---")
        pit_cost = calculate_pit_stop_time(laps_data, track_name)
        results[track] = pit_cost
    
    return results

def calculate_degradation_rate_v2(laps_data):
    # (이전과 동일한 함수 내용)
    base_rates = {
        'SOFT': 0.035, 'MEDIUM': 0.022, 'HARD': 0.013,
        'INTERMEDIATE': 0.045, 'WET': 0.080
    }
    # ... (함수 내용 전체) ...
    tyre_data = laps_data.dropna(subset=['LapTime', 'TyreLife', 'Compound'])
    degradation_rates = {}
    
    for compound in tyre_data['Compound'].unique():
        if compound not in base_rates:
            continue
        compound_data = tyre_data[tyre_data['Compound'] == compound]
        if len(compound_data) > 20:
            intervals = []
            for start_life in range(1, 31, 5):
                interval_data = compound_data[
                    (compound_data['TyreLife'] >= start_life) & 
                    (compound_data['TyreLife'] < start_life + 5)
                ]
                if len(interval_data) > 0:
                    intervals.append({
                        'start_life': start_life,
                        'avg_time': interval_data['LapTime'].median()
                    })
            if len(intervals) >= 3:
                time_increases = []
                for i in range(1, len(intervals)):
                    prev_time = intervals[i-1]['avg_time']
                    curr_time = intervals[i]['avg_time']
                    increase_rate = (curr_time - prev_time) / prev_time
                    time_increases.append(increase_rate)
                if time_increases:
                    degradation_rate = np.mean(time_increases) * 5
                    degradation_rates[compound] = max(0.005, min(0.08, abs(degradation_rate)))
                else:
                    degradation_rates[compound] = base_rates[compound]
            else:
                degradation_rates[compound] = base_rates[compound]
        else:
            degradation_rates[compound] = base_rates[compound]

    for compound in base_rates:
        if compound not in degradation_rates:
            degradation_rates[compound] = base_rates[compound]
    
    # 드라이 타이어에 대해서만 순서 보장
    dry_compounds = ['HARD', 'MEDIUM', 'SOFT']
    dry_rates = [degradation_rates.get(c) for c in dry_compounds if c in degradation_rates]
    
    if len(dry_rates) == 3: # 드라이 컴파운드가 모두 있을 때만 정렬
        dry_rates.sort()
        for i, compound in enumerate(dry_compounds):
            degradation_rates[compound] = dry_rates[i]
    
    return degradation_rates


def calculate_stint_params_proper(laps_data):
    # (이전과 동일한 함수 내용 및 의존 함수들 포함)
    # ... (filter_performance_based_stints, analyze_stint_distribution, validate_stint_logic 등)
    pit_data = laps_data.dropna(subset=['PitInTime', 'LapNumber', 'Driver', 'Compound'])
    pit_data = pit_data.sort_values(['Driver', 'Time'])
    pit_data['NextPitLap'] = pit_data.groupby('Driver')['LapNumber'].shift(-1)
    pit_data['StintLength'] = pit_data['NextPitLap'] - pit_data['LapNumber']
    stint_data = pit_data[pit_data['StintLength'] > 0].dropna(subset=['StintLength', 'Compound'])
    
    optimal_stint = {}
    min_stint = {}
    
    performance_based_stints = filter_performance_based_stints(stint_data, laps_data)
    
    theoretical_values = {
        'SOFT': 16, 'MEDIUM': 25, 'HARD': 35,
        'INTERMEDIATE': 20, 'WET': 12
    }
    
    # 모든 컴파운드에 대해 계산
    all_compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
    for compound in all_compounds:
        if compound not in laps_data['Compound'].unique():
            continue
            
        compound_stints = performance_based_stints.get(compound, [])
        
        if len(compound_stints) >= 5:
            stint_analysis = analyze_stint_distribution(compound_stints)
            optimal_stint[compound] = stint_analysis['recommended']
        else:
            optimal_stint[compound] = theoretical_values[compound]
        
        min_stint[compound] = 8 if compound in ['SOFT', 'MEDIUM', 'HARD'] else 5
        print(f"{compound}: {len(compound_stints)} quality stints, optimal={optimal_stint.get(compound)} laps")
    
    validated_stints = validate_stint_logic(optimal_stint, theoretical_values)
    
    return validated_stints, min_stint


def filter_performance_based_stints(stint_data, laps_data):
    """
    성능 기반으로 유효한 stint만 필터링
    - 전략적/비정상적 stint 제외
    - 타이어별 특성을 고려한 필터링
    """
    filtered_stints = {}
    
    for compound in stint_data['Compound'].unique():
        compound_stints = stint_data[stint_data['Compound'] == compound].copy()
        
        # 1. 기본 범위 필터링
        if compound == 'SOFT':
            valid_range = (8, 25)  # 소프트는 짧은 범위
        elif compound == 'MEDIUM':
            valid_range = (12, 35) # 미디엄은 중간 범위
        else:  # HARD
            valid_range = (18, 45) # 하드는 긴 범위
        
        in_range_stints = compound_stints[
            (compound_stints['StintLength'] >= valid_range[0]) &
            (compound_stints['StintLength'] <= valid_range[1])
        ]
        
        # 2. 성능 기반 필터링 (레이스 페이스가 정상적인 stint만)
        performance_filtered = filter_by_race_performance(in_range_stints, laps_data, compound)
        
        # 3. 통계적 이상치 제거
        if len(performance_filtered) > 10:
            Q1 = performance_filtered['StintLength'].quantile(0.25)
            Q3 = performance_filtered['StintLength'].quantile(0.75)
            IQR = Q3 - Q1
            
            final_stints = performance_filtered[
                (performance_filtered['StintLength'] >= Q1 - 1.5 * IQR) &
                (performance_filtered['StintLength'] <= Q3 + 1.5 * IQR)
            ]
        else:
            final_stints = performance_filtered
        
        # final_stints가 비어있지 않을 때만 컬럼에 접근
        if not final_stints.empty:
            filtered_stints[compound] = final_stints['StintLength'].tolist()
        else:
            filtered_stints[compound] = []
        
        print(f"{compound}: {len(compound_stints)} → {len(final_stints)} stints after filtering")
    
    return filtered_stints



def filter_by_race_performance(stint_data, laps_data, compound):
    """
    레이스 성능을 고려한 stint 필터링
    - 비정상적으로 느린 페이스의 stint 제외
    - Safety Car, 트래픽 등으로 인한 왜곡 제거
    """
    if stint_data.empty:
        return stint_data
    
    # 해당 컴파운드의 정상 성능 범위 계산
    compound_laps = laps_data[laps_data['Compound'] == compound]
    if len(compound_laps) < 50:
        return stint_data  # 데이터 부족시 필터링 건너뛰기
    
    # 정상 랩타임 범위 (25-75% 백분위수)
    normal_lap_range = (
        compound_laps['LapTime'].quantile(0.25),
        compound_laps['LapTime'].quantile(0.75)
    )
    
    valid_stints = []
    
    for _, stint_row in stint_data.iterrows():
        driver = stint_row['Driver']
        start_lap = stint_row['LapNumber']
        end_lap = start_lap + stint_row['StintLength']
        
        # 해당 stint의 랩타임들 조회
        stint_laps = laps_data[
            (laps_data['Driver'] == driver) &
            (laps_data['LapNumber'] >= start_lap) &
            (laps_data['LapNumber'] < end_lap) &
            (laps_data['Compound'] == compound)
        ]
        
        if len(stint_laps) >= stint_row['StintLength'] * 0.7:  # 70% 이상 데이터 있음
            # stint 평균 페이스가 정상 범위에 있는지 확인
            stint_avg_time = stint_laps['LapTime'].median()
            
            if normal_lap_range[0] <= stint_avg_time <= normal_lap_range[1] * 1.1:  # 약간의 여유
                valid_stints.append(stint_row)
    
    return pd.DataFrame(valid_stints) if valid_stints else pd.DataFrame()


def analyze_stint_distribution(stint_lengths):
    """
    stint 길이 분포 분석으로 최적값 도출
    """
    stint_array = np.array(stint_lengths)
    
    analysis = {
        'mean': np.mean(stint_array),
        'median': np.median(stint_array),
        'mode': float(sps.mode(stint_array, keepdims=False)[0]) if len(stint_array) > 0 else np.median(stint_array),
        'p75': np.percentile(stint_array, 75),  # 상위 25% 평균
        'recommended': 0
    }
    
    # 다양한 통계값의 가중평균으로 추천값 계산
    # 중간값(40%) + 75%ile(30%) + 평균(20%) + 최빈값(10%)
    analysis['recommended'] = int(
        analysis['median'] * 0.4 +
        analysis['p75'] * 0.3 +
        analysis['mean'] * 0.2 +
        analysis['mode'] * 0.1
    )
    
    return analysis


def validate_stint_logic(optimal_stint, theoretical_values):
    """
    논리적 검증 및 미세 조정 (강제 변경 대신 경고 및 보정)
    """
    validated = optimal_stint.copy()
    adjustments_made = False
    
    # 검증 1: 기본 순서 확인
    if validated.get('SOFT', 0) >= validated.get('MEDIUM', 0):
        print(f"⚠️  WARNING: SOFT stint ({validated.get('SOFT')}) >= MEDIUM stint ({validated.get('MEDIUM')})")
        
        # 데이터 신뢰도 기반 조정
        if 'SOFT' in validated and 'MEDIUM' in validated:
            # 이론값과의 편차가 큰 쪽을 이론값에 가깝게 조정
            soft_deviation = abs(validated['SOFT'] - theoretical_values['SOFT'])
            medium_deviation = abs(validated['MEDIUM'] - theoretical_values['MEDIUM'])
            
            if soft_deviation > medium_deviation:
                validated['SOFT'] = int((validated['SOFT'] + theoretical_values['SOFT']) / 2)
                adjustments_made = True
                print(f"   → SOFT adjusted to {validated['SOFT']}")
    
    # 검증 2: MEDIUM vs HARD
    if validated.get('MEDIUM', 0) >= validated.get('HARD', 0):
        print(f"⚠️  WARNING: MEDIUM stint ({validated.get('MEDIUM')}) >= HARD stint ({validated.get('HARD')})")
        
        if 'MEDIUM' in validated and 'HARD' in validated:
            medium_deviation = abs(validated['MEDIUM'] - theoretical_values['MEDIUM'])
            hard_deviation = abs(validated['HARD'] - theoretical_values['HARD'])
            
            if medium_deviation > hard_deviation:
                validated['MEDIUM'] = int((validated['MEDIUM'] + theoretical_values['MEDIUM']) / 2)
                adjustments_made = True
                print(f"   → MEDIUM adjusted to {validated['MEDIUM']}")
            else:
                validated['HARD'] = int((validated['HARD'] + theoretical_values['HARD']) / 2)
                adjustments_made = True
                print(f"   → HARD adjusted to {validated['HARD']}")
    
    if adjustments_made:
        print("✅ Logical adjustments completed based on data quality")
    else:
        print("✅ All stint values follow logical order")
    
    return validated


def calculate_temp_threshold(laps_data):
    """
    TrackTemp와 LapTime 간 상관관계를 분석해 temp_threshold 계산
    - 최소 포인트 수 20 필터링
    """
    temp_data = laps_data.dropna(subset=['LapTime', 'TrackTemp', 'Compound'])
    temp_bins = np.linspace(temp_data['TrackTemp'].min(), temp_data['TrackTemp'].max(), 10)
    
    temp_threshold = {}
    for compound in temp_data['Compound'].unique():
        compound_data = temp_data[temp_data['Compound'] == compound].copy()
        compound_data.loc[:, 'TempBin'] = pd.cut(compound_data['TrackTemp'], bins=temp_bins, labels=[i for i in range(len(temp_bins)-1)])
        
        temp_means = compound_data.groupby('TempBin', observed=True).agg({'LapTime': 'mean'})
        if temp_means.empty:
            temp_threshold[compound] = compound_data['TrackTemp'].mean()
        else:
            valid_bins = compound_data['TempBin'].value_counts()[compound_data['TempBin'].value_counts() >= 20].index
            temp_means = temp_means.loc[valid_bins]
            if not temp_means.empty:
                base_mean = temp_means['LapTime'].min()
                max_increase_idx = (temp_means['LapTime'] - base_mean > base_mean * 0.1).idxmax()
                if max_increase_idx in valid_bins:
                    temp_threshold[compound] = (temp_bins[max_increase_idx] + temp_bins[max_increase_idx + 1]) / 2
                else:
                    temp_threshold[compound] = compound_data['TrackTemp'].mean()
            else:
                temp_threshold[compound] = compound_data['TrackTemp'].mean()
        print(f"{compound}: Calculated temp_threshold = {temp_threshold[compound]:.1f}°C, Temp means = {temp_means['LapTime'].to_dict()}, Points per bin = {compound_data['TempBin'].value_counts().to_dict()}")
    
    return temp_threshold


def calculate_base_pace(laps_data):
    """
    초기 스틴트의 평균 LapTime을 이용해 base_pace 계산
    - 모든 컴파운드(WET, INTERMEDIATE 포함)를 처리하도록 수정됨
    """
    pace_data = laps_data.dropna(subset=['LapTime', 'Compound', 'TyreLife'])
    # TyreLife가 1 이하인 랩만 필터링하여 각 타이어의 초기 성능을 확인
    pace_data = pace_data[pace_data['TyreLife'] <= 1]
    
    base_paces = {}
    
    # 데이터에 존재하는 모든 고유 컴파운드에 대해 반복
    for compound in pace_data['Compound'].unique():
        compound_mean = pace_data[pace_data['Compound'] == compound]['LapTime'].mean()
        
        # 유효한 랩타임이 있는 경우에만 pace를 계산
        if pd.notna(compound_mean) and compound_mean > 0:
            if compound == 'SOFT':
                base_paces[compound] = 1.0  # 기준 속도
            elif compound == 'MEDIUM':
                base_paces[compound] = 0.98 # SOFT보다 약간 느림
            elif compound == 'HARD':
                base_paces[compound] = 0.96 # MEDIUM보다 약간 느림
            elif compound == 'INTERMEDIATE':
                base_paces[compound] = 0.85 # 드라이 타이어보다 확연히 느림
            elif compound == 'WET':
                base_paces[compound] = 0.75 # 가장 느림
            else:
                # 'Unknown' 등 예상치 못한 컴파운드에 대한 안전장치
                base_paces[compound] = 0.70 
        
        # base_paces 딕셔너리에 해당 compound가 있는지 확인 후 출력 (KeyError 방지)
        if compound in base_paces:
            print(f"{compound} initial data points: {len(pace_data[pace_data['Compound'] == compound])}, base_pace = {base_paces[compound]}")
        else:
            # pace가 계산되지 않은 경우 (e.g., 유효한 랩타임이 없는 경우)
            print(f"{compound} initial data points: {len(pace_data[pace_data['Compound'] == compound])}, base_pace calculation skipped.")

    print("\nCalculated base_pace:", base_paces)
    return base_paces


def calculate_all_compound_performance(laps_data):
    """
    주어진 랩 데이터에 대해 모든 통계 기반 컴파운드 성능 파라미터를 계산합니다.
    
    Parameters:
    -----------
    laps_data : pd.DataFrame
        분석할 랩 데이터 (Dry 또는 Wet으로 필터링된 데이터)
        
    Returns:
    --------
    dict
        각 컴파운드별 성능 파라미터(base_pace, degradation_rate 등)를 포함하는 
        중첩된 딕셔너리.
    """
    print("\n=== 컴파운드 성능 파라미터 계산 시작 ===")
    
    # 1. 각 파라미터 계산
    base_paces = calculate_base_pace(laps_data)
    degradation_rates = calculate_degradation_rate_v2(laps_data)
    optimal_stint, min_stint = calculate_stint_params_proper(laps_data)
    temp_threshold = calculate_temp_threshold(laps_data)

    # 2. 결과 통합
    compound_performance = {}
    
    # laps_data에 존재하는 고유한 컴파운드 목록에 대해서만 반복
    existing_compounds = laps_data['Compound'].unique()
    
    for compound in existing_compounds:
        # 계산된 값이 없는 경우를 대비하여 기본값(get 메서드의 두 번째 인자) 설정
        compound_performance[compound] = {
            "base_pace": base_paces.get(compound, 0.95),
            "degradation_rate": degradation_rates.get(compound, 0.05),
            "optimal_stint": optimal_stint.get(compound, 15),
            "min_stint": min_stint.get(compound, 10),
            "temp_threshold": temp_threshold.get(compound, 28.3)
        }
    
    print("\n=== 컴파운드 성능 파라미터 계산 완료 ===")
    print(compound_performance)
    
    return compound_performance

def select_next_compound(current_compound, used_compounds, rainfall, track_temp, current_lap, race_laps, pit_count, params):
    """
    다음으로 사용할 타이어 컴파운드를 선택하는 헬퍼 함수
    - 현재 날씨와 레이스 상황에 맞는 파라미터(params)를 사용
    """
    remaining_laps = race_laps - current_lap
    
    # 1. 날씨 기반 선택 (최우선)
    if rainfall > 0.1 and 'wet_strategy_params' in params:
        # 비가 오면 Wet 파라미터를 사용해야 함
        wet_compounds = params['wet_strategy_params']['compound_performance']
        if "WET" in wet_compounds and rainfall > 2.0: # 강한 비
             return "WET"
        if "INTERMEDIATE" in wet_compounds:
            return "INTERMEDIATE"

    # 2. Dry 날씨 기반 선택
    # 사용 가능한 Dry 타이어
    dry_params = params['dry_strategy_params']
    available_dry = {"SOFT", "MEDIUM", "HARD"} - used_compounds
    
    # 파라미터에서 t1, t2, t3 임계값 가져오기
    thresholds = dry_params['compound_choice_thresholds']
    t1, t2, t3 = thresholds['t1'], thresholds['t2'], thresholds['t3']
    
    if track_temp > t1 and "HARD" in available_dry:
        return "HARD"
    elif remaining_laps > t2 and "HARD" in available_dry:
        return "HARD"
    elif remaining_laps > t3 and "MEDIUM" in available_dry:
        return "MEDIUM"
    
    # Fallback 로직: 가능한 타이어 중 가장 내구성이 좋은 순서대로 선택
    if "SOFT" in available_dry and pit_count == 0:
        return "SOFT"
    if "MEDIUM" in available_dry:
        return "MEDIUM"
    if "HARD" in available_dry:
        return "HARD"
        
    # 모든 Dry 타이어를 이미 사용했다면, 가장 내구성이 좋은 타이어 중 하나를 다시 선택
    return "MEDIUM" if "MEDIUM" in dry_params['compound_performance'] else "HARD"


def run_race_simulation(lap_data_df, params_data, initial_tyre="HARD", race_laps=58):
    """
    전체 레이스를 시뮬레이션하고 최적의 피트 스톱 전략을 도출합니다.
    - 날씨에 따라 동적으로 파라미터를 선택합니다.
    """
    df_race = lap_data_df[lap_data_df['lap'] <= race_laps].copy()

    print(f"Race simulation started: Laps 1-{race_laps}")
    print(f"Initial compound: {initial_tyre}")
    
    strategy = []
    used_compounds = {initial_tyre}
    current_compound = initial_tyre
    last_pit_lap = 0
    current_stint_length = 0
    total_time = 0.0
    pit_count = 0
    
    for _, row in lap_data_df.iterrows():
        lap_num = int(row['lap'])
        current_stint_length += 1
        
        # --- ⭐️ 동적 파라미터 선택 (핵심 수정) ⭐️ ---
        is_wet = row['rainfall'] > 0
        if is_wet and 'wet_strategy_params' in params_data:
            active_params = params_data['wet_strategy_params']
        else:
            active_params = params_data['dry_strategy_params']
        
        # 현재 컴파운드가 active_params에 없으면 Dry 파라미터로 fallback
        if current_compound not in active_params['compound_performance']:
            active_params = params_data['dry_strategy_params']
            
        perf = active_params['compound_performance'][current_compound]
        pit_cost = active_params['pit_stop_time']
        
        # 랩타임 계산
        degradation = 1 + (perf["degradation_rate"] * current_stint_length / 100)
        adjusted_laptime = row['laptime'] * perf["base_pace"] * degradation
        total_time += adjusted_laptime
        
        # 피트스톱 결정 로직
        should_pit = False
        pit_reason = ""
        
        # Dry/Wet 공통 조건: 최소 스틴트 길이 충족 및 최대 피트 횟수 미만
        if lap_num - last_pit_lap >= perf.get("min_stint", 5) and pit_count < 3:
            if not is_wet: # Dry 로직
                if (row['pit_proba'] > active_params['pit_proba_threshold'] and \
                    current_stint_length >= perf["optimal_stint"] * 0.9) or \
                   (row['tyre_wear'] > active_params['TyreLife_threshold']):
                    should_pit = True
                    pit_reason = f"ML-Predict(Prob:{row['pit_proba']:.2f}, Wear:{row['tyre_wear']:.1f})"
            else: # Wet 로직
                if current_stint_length >= perf["optimal_stint"]:
                    should_pit = True
                    pit_reason = f"Wet-Stint({current_stint_length}/{perf['optimal_stint']})"

            # 날씨 변화로 인한 강제 피트스톱
            if is_wet and current_compound in ["SOFT", "MEDIUM", "HARD"]:
                should_pit = True
                pit_reason = f"RAIN! Change to Wet-Tyre"
            elif not is_wet and current_compound in ["INTERMEDIATE", "WET"]:
                 should_pit = True
                 pit_reason = f"TRACK DRY! Change to Dry-Tyre"
        
        if should_pit:
            total_time += pit_cost
            next_compound = select_next_compound(current_compound, used_compounds, row['rainfall'], 
                                                 row['track_temp'], lap_num, race_laps, pit_count, params_data)
            
            strategy.append({
                "lap": lap_num, "from_tyre": current_compound, "to_tyre": next_compound,
                "reason": pit_reason, "stint_length": current_stint_length
            })
            print(f"Lap {lap_num}: PIT! {current_compound}->{next_compound} ({pit_reason})")
            
            used_compounds.add(next_compound)
            current_compound = next_compound
            last_pit_lap = lap_num
            current_stint_length = 0
            pit_count += 1
    
    print("\nRace simulation complete:")
    print(f"- Total race time: {total_time/60:.2f} minutes")
    return total_time, strategy

# 데이터 전처리 및 정렬 (수정된 부분)
def fix_time_sorting(df, time_col='Time', group_col='GrandPrix'):
    """
    시간 컬럼의 정렬 문제를 수정하는 함수
    """
    # NaN 값 제거
    df = df.dropna(subset=[time_col]).copy()
    
    # datetime으로 변환
    df[time_col] = pd.to_datetime(df[time_col])
    
    # 각 그룹별로 정렬 확인 및 수정
    fixed_groups = []
    
    for group_name in df[group_col].unique():
        group_data = df[df[group_col] == group_name].copy()
        
        # 시간순으로 정렬
        group_data = group_data.sort_values(time_col).reset_index(drop=True)
        
        # 정렬 확인
        if not group_data[time_col].is_monotonic_increasing:
            print(f"Warning: {group_name} has non-monotonic time values, fixing...")
            # 중복된 시간값이 있을 경우 미세하게 조정
            duplicated = group_data[time_col].duplicated()
            if duplicated.any():
                print(f"Found {duplicated.sum()} duplicate time values in {group_name}")
                # 중복된 시간에 microsecond 추가
                for i, is_dup in enumerate(duplicated):
                    if is_dup:
                        group_data.iloc[i, group_data.columns.get_loc(time_col)] += pd.Timedelta(microseconds=i)
        
        fixed_groups.append(group_data)
        print(f"Fixed {group_name}: {len(group_data)} rows, time range: {group_data[time_col].min()} to {group_data[time_col].max()}")
    
    return pd.concat(fixed_groups, ignore_index=True)
def evaluate_strategy(lap_data_df, params_data, strategy_plan, initial_tyre, race_laps):
    # strategy_plan 예시: [{'lap': 15, 'tyre': 'HARD'}, {'lap': 30, 'tyre': 'MEDIUM'}]
    
    total_time = 0.0
    current_compound = initial_tyre
    last_pit_lap = 0
    
    pit_stops = {s['lap']: s['tyre'] for s in strategy_plan}

    for _, row in lap_data_df.iterrows():
        lap_num = int(row['lap'])
        
        # 피트 스톱 수행
        if lap_num in pit_stops:
            total_time += params_data['dry_strategy_params']['pit_stop_time']
            current_compound = pit_stops[lap_num]
            last_pit_lap = lap_num

        stint_length = lap_num - last_pit_lap
        
        # 랩타임 계산 (기존 로직과 거의 동일)
        is_wet = row['rainfall'] > 0
        params_key = 'wet_strategy_params' if is_wet and 'wet_strategy_params' in params_data else 'dry_strategy_params'
        
        if current_compound not in params_data[params_key]['compound_performance']:
             params_key = 'dry_strategy_params' # Fallback
             
        perf = params_data[params_key]['compound_performance'][current_compound]
        degradation = 1 + (perf["degradation_rate"] * stint_length / 100)
        adjusted_laptime = row['laptime'] * perf["base_pace"] * degradation
        total_time += adjusted_laptime
        
    return total_time

def generate_candidate_strategies(total_laps):
    """총 랩 수를 바탕으로 1스톱, 2스톱 전략 후보들을 동적으로 생성합니다."""
    strategies = {}
    
    # --- 1-Stop 전략 후보 ---
    # M -> H (가장 일반적)
    pit_lap_1s_a = int(total_laps * 0.4)
    strategies[f"1-Stop (M-H, Lap {pit_lap_1s_a})"] = [{'lap': pit_lap_1s_a, 'tyre': 'HARD'}]
    
    # S -> H (공격적)
    pit_lap_1s_b = int(total_laps * 0.3)
    strategies[f"1-Stop (S-H, Lap {pit_lap_1s_b})"] = [{'lap': pit_lap_1s_b, 'tyre': 'HARD'}]

    # --- 2-Stop 전략 후보 ---
    # S -> M -> M
    pit_lap_2s_a1 = int(total_laps / 3)
    pit_lap_2s_a2 = int(total_laps * 2 / 3)
    strategies[f"2-Stop (S-M-M, Laps {pit_lap_2s_a1}, {pit_lap_2s_a2})"] = [
        {'lap': pit_lap_2s_a1, 'tyre': 'MEDIUM'}, 
        {'lap': pit_lap_2s_a2, 'tyre': 'MEDIUM'}
    ]
    
    # S -> M -> S
    pit_lap_2s_b1 = int(total_laps * 0.25)
    pit_lap_2s_b2 = int(total_laps * 0.65)
    strategies[f"2-Stop (S-M-S, Laps {pit_lap_2s_b1}, {pit_lap_2s_b2})"] = [
        {'lap': pit_lap_2s_b1, 'tyre': 'MEDIUM'},
        {'lap': pit_lap_2s_b2, 'tyre': 'SOFT'}
    ]
    
    return strategies



In [ ]:
# Optuna 목표 함수 정의
def objective(trial):
    # 하이퍼파라미터 검색 공간
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42,
        'objective': 'reg:squarederror'
    }

    # --- 다중 출력 모델 학습 (변환된 데이터 사용) ---
    reg_param = param.copy()
    reg_param['objective'] = 'reg:squarederror'
    multi_output_model = xgb.XGBRegressor(**reg_param)
    # X_train이 아닌 X_train_transformed를 사용
    multi_output_model.fit(X_train_transformed, y_multi_train[:, [0, 2]])

    # --- PitStop 분류 모델 학습 (변환 및 오버샘플링된 데이터 사용) ---
    clf_param = param.copy()
    clf_param['objective'] = 'binary:logistic'
    clf_param['scale_pos_weight'] = trial.suggest_float('scale_pos_weight', 3, 5)
    xgb_pitstop = xgb.XGBClassifier(**clf_param)
    xgb_pitstop.fit(X_train_pit_res, y_pitstop_train_res)

    # --- 예측 (변환된 테스트 데이터 사용) ---
    y_pred_multi = multi_output_model.predict(X_test_transformed)
    y_pred_pitstop_proba = xgb_pitstop.predict_proba(X_test_transformed)[:, 1]

    # --- 평가 ---
    y_pred_laptime = y_pred_multi[:, 0]
    y_pred_tyrewear = y_pred_multi[:, 1] # XGBoost MultiOutput은 순서대로 출력하므로 인덱스 1
    
    fpr, tpr, thresholds = roc_curve(y_pitstop_test, y_pred_pitstop_proba)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx] if len(thresholds) > 0 else 0.5
    y_pred_pitstop = (y_pred_pitstop_proba >= optimal_threshold).astype(int)

    precision = precision_score(y_pitstop_test, y_pred_pitstop, zero_division=0)
    f1 = f1_score(y_pitstop_test, y_pred_pitstop, zero_division=0)
    r2_laptime = r2_score(y_laptime_test, y_pred_laptime)
    r2_tyre_wear = r2_score(y_tyrewear_test, y_pred_tyrewear)
    
    # 가중치 합산 점수 반환
    score = 0.4 * f1 + 0.2 * precision + 0.2 * r2_laptime + 0.2 * r2_tyre_wear
    return score


# --- 3. 적합도 평가 함수 (이전과 동일) ---
def evaluate_hybrid_strategy(individual):
    """
    GA가 제안한 전략(individual)의 '총 시간'과 '리스크'를 모두 평가합니다.
    """
    MINIMUM_STINT_LAPS = 7  # 모든 스틴트의 최소 랩 수 (이 값은 조절 가능)


    # 1. GA의 염색체를 시뮬레이션용 plan으로 변환
    pit1_lap, pit1_tyre_idx, pit2_lap, pit2_tyre_idx = individual
    plan = []
    if pit1_lap > 0: plan.append({'lap': int(pit1_lap), 'tyre': COMPOUNDS[pit1_tyre_idx]})
    if pit2_lap > 0: plan.append({'lap': int(pit2_lap), 'tyre': COMPOUNDS[pit2_tyre_idx]})
    
    # --- ⭐️ 여기가 핵심 수정 부분입니다 (최소 스틴트 길이 검사) ⭐️ ---
    # 2. 모든 스틴트의 길이가 최소 기준을 만족하는지 검사
    stint_lengths = []
    last_pit_lap = 0
    for pit in plan:
        stint_lengths.append(pit['lap'] - last_pit_lap)
        last_pit_lap = pit['lap']
    stint_lengths.append(race_laps - last_pit_lap) # 마지막 스틴트

    for length in stint_lengths:
        if length < MINIMUM_STINT_LAPS:
            # 스틴트 길이가 너무 짧으면 실격 처리 (아주 큰 페널티)
            return (999999,)

    # 2. F1의 '2가지 컴파운드 사용' 규정 검사
    # (시작 타이어는 INITIAL_COMPOUND 변수를 사용한다고 가정)
    compounds_used = {initial_compound}
    for pit in plan:
        compounds_used.add(pit['tyre'])
    
    # 사용된 컴파운드 종류가 2가지 미만이면 엄청난 페널티를 부여하고 즉시 반환
    if len(compounds_used) < 2:
        # 이 전략은 실격(Disqualified)이므로, 아주 큰 시간(페널티)을 반환하여 도태시킴
        return (999999,) 


    # 2. '총 레이스 시간' 계산 (기존과 동일)
    # evaluate_strategy 함수는 이제 '헬퍼' 함수가 됩니다.
    total_time = evaluate_strategy(lap_data_df, params_data, plan, initial_compound, race_laps)

    # 3. '리스크 패널티' 계산 (반응형 모델 인수 사용)
    risk_penalty = 0
    dry_params = params_data['dry_strategy_params'] # 결정용 파라미터 로드
    
    for pit_stop in plan:
        pit_lap = pit_stop['lap']
        
        # 해당 랩의 데이터(ML 예측치) 가져오기
        lap_info = lap_data_df[lap_data_df['lap'] == pit_lap]
        if not lap_info.empty:
            wear_at_pit = lap_info.iloc[0]['tyre_wear']
            proba_at_pit = lap_info.iloc[0]['pit_proba']

            # 패널티 조건 1: 타이어를 너무 한계까지 몰아붙인 전략에 큰 패널티
            if wear_at_pit > dry_params['TyreLife_threshold'] * 1.5: # 임계값의 150% 초과 시
                penalty = (wear_at_pit - dry_params['TyreLife_threshold']) * 5 # 초과한 만큼 큰 패널티
                risk_penalty += penalty
                print(f"DEBUG: Lap {pit_lap} - High Wear Penalty! Wear:{wear_at_pit:.1f} -> Penalty:{penalty:.1f}s")

            # 패널티 조건 2: ML모델이 전혀 피트인을 예측하지 않은 랩에 들어가는 전략에 작은 패널티
            if proba_at_pit < dry_params['pit_proba_threshold'] * 0.1: # 임계값의 10% 미만 시
                penalty = 10 # 10초 고정 패널티
                risk_penalty += penalty
                print(f"DEBUG: Lap {pit_lap} - Low Proba Penalty! Proba:{proba_at_pit:.2f} -> Penalty:{penalty:.1f}s")

    # 최종 적합도 = 총 시간 + 리스크 패널티
    final_score = total_time + risk_penalty
    return (final_score,) # DEAP는 튜플로 반환해야 함


def create_random_strategy(race_laps):
    num_stops = random.randint(1, 2)
    
    if num_stops == 1:
        pit1_lap = random.randint(10, race_laps - 10)
        pit1_tyre = random.randint(0, 2)
        # ⭐️ 그냥 리스트가 아닌, creator.Individual로 감싸서 반환
        return creator.Individual([pit1_lap, pit1_tyre, 0, 0]) 
    else: # 2 stops
        pit1_lap = random.randint(10, int(race_laps / 2))
        pit2_lap = random.randint(pit1_lap + 10, race_laps - 10)
        pit1_tyre = random.randint(0, 2)
        pit2_tyre = random.randint(0, 2)
        # ⭐️ 여기도 creator.Individual로 감싸서 반환
        return creator.Individual([pit1_lap, pit1_tyre, pit2_lap, pit2_tyre])

# --- 4. ⭐️ 규칙을 준수하는 스마트한 교배, 돌연변이, 선택 연산자 정의 ⭐️ ---
def custom_mutate(individual, low, up, indpb):
    # DEAP의 기본 돌연변이 함수를 먼저 실행
    tools.mutUniformInt(individual, low, up, indpb)
    # ⭐️ 핵심 수정: 두 피트스톱 랩이 같아지는 경우를 처리하는 로직 추가 ⭐️
    if individual[2] > 0: # 2스톱 전략일 경우에만 검사
        if individual[0] == individual[2]:
            # 두 랩이 같으면, 비현실적이므로 2스톱을 1스톱으로 변경
            individual[2] = 0
            individual[3] = 0
        elif individual[0] > individual[2]:
            # 순서가 어긋났으면 바로잡음
            individual[0], individual[2] = individual[2], individual[0]
    return individual, # 튜플로 반환해야 함

def custom_mate(ind1, ind2):
    # DEAP의 기본 교배 함수를 먼저 실행
    tools.cxTwoPoint(ind1, ind2)
    # ⭐️ 핵심 수정: 교배 후 생성된 두 자식 전략 모두에 대해 검사 ⭐️
    for ind in [ind1, ind2]:
        if ind[2] > 0: # 2스톱 전략일 경우에만 검사
            if ind[0] == ind[2]:
                # 두 랩이 같으면, 1스톱으로 변경
                ind[2] = 0
                ind[3] = 0
            elif ind[0] > ind[2]:
                # 순서가 어긋났으면 바로잡음
                ind[0], ind[2] = ind[2], ind[0]
    return ind1, ind2


# ------------------------------------------------------------------------------
# [ STEP 3: 데이터 및 파라미터 로딩 ]
# ------------------------------------------------------------------------------

track = [
    "Eifel",
    "Emilia_Romagna",
    "French",
    "German",
    "Hungarian",
    "Italian",
    "Japanese",
    "Las_Vegas",
    "Mexico_City",
    "Miami",
    "Monaco",
    "Portuguese",
    "Qatar",
    "Russian",
    "Sakhir",
    "Saudi_Arabian",
    "Singapore",
    "Spanish",
    "Styrian",
    "Sao_Paulo",
    "Turkish",
    "Tuscan",
    "United_States"
]
compound = ["HARD", "MEDIUM", "SOFT"]

# track_name = 'Australian' # <--- 분석하고 싶은 트랙 이름만 여기서 변경하세요.
# initial_compound = "D" # 시뮬레이션 시작 타이어 (필요시 변경)

for track_name in track :


    print(f">>> STEP 3: '{track_name}' 데이터 및 파라미터 로딩 시작...")

    # 트랙 설정 파일 로드
    with open('/home/azureuser/cloudfiles/code/Users/data/tracks_config.json', 'r', encoding='utf-8') as f:
        tracks_config = json.load(f)
    if track_name not in tracks_config:
        raise ValueError(f"'{track_name}' 설정이 'tracks_config.json'에 없습니다.")

    # 타겟 트랙의 정보 추출
    track_info = tracks_config[track_name]
    race_laps = track_info['total_laps']
    target_group = track_info.get('group') # 타겟 트랙의 그룹 이름 가져오기

    # grand_prix_list 생성
    grand_prix_list = []
    if target_group:
        print(f"'{track_name}' 트랙은 '{target_group}' 그룹에 속합니다. 그룹 전체 데이터를 로드합니다.")
        # 전체 설정 파일에서 같은 그룹에 속한 모든 트랙을 찾음
        for t_name, t_info in tracks_config.items():
            if t_info.get('group') == target_group:
                grand_prix_list.extend(t_info['data_files']) # 해당 트랙의 데이터 파일 목록을 추가
    else:
        # 그룹이 지정되지 않은 경우, 해당 트랙의 데이터만 사용
        print(f"'{track_name}' 트랙에 그룹이 지정되지 않았습니다. 단일 트랙 데이터만 로드합니다.")
        grand_prix_list = track_info['data_files']

    print(f"데이터 로딩 대상: {grand_prix_list}")


    # Azure Blob Storage 설정
    azure_access = AzureStorageAccess()
    # 모든 파일 순회하고 싶다.
    # for file in azure_access.get_all_file():
    #     file_name = file.name
    #     data_frame = azure_access.get_file_by_data_frame(file_name)   <- 지금 돌고있는 파일블롭을 바로 데이터로 변환

    #       data_frame = azure_access.read_csv_by_data_frame(file_name) <- 지금 돌고있는 파일블롭의 이름을 뽑아서 다시 파일 전부 돌아가며 이름과 똑같은 파일찾아서 반환


    all_laps_data = []
    all_weather_data = []

    compound_map = {
        'SOFT': 'SOFT',
        'SUPERSOFT': 'SOFT',
        'ULTRASOFT': 'SOFT',
        'HYPERSOFT': 'SOFT',
        'MEDIUM': 'MEDIUM',
        'HARD': 'HARD',
        'INTERMEDIATE': 'INTERMEDIATE',   # 비건조
        'WET': 'WET',                     # 비건조
        'TEST': 'TEST',
        'TEST_UNKNOWN': 'Unknown',
        'UNKNOWN': 'Unknown'
    }

    for gp in grand_prix_list:
        session_info = azure_access.read_csv_by_data_frame(f'{gp}/session_info.csv')
        if session_info is not None:
            print(f"Loaded {gp}/session_info.csv with columns: {session_info.columns.tolist()}")
            base_time = pd.to_datetime(session_info[session_info['Type'] == 'Race']['StartDate'].iloc[0])
        else:
            year = gp.split('/')[0]
            base_time = pd.to_datetime(f'{year}-12-08 14:00:00')
            print(f"Warning: Failed to load {gp}/session_info.csv, using default base_time: {base_time}")
        
        laps_data = azure_access.read_csv_by_data_frame(f'{gp}/laps.csv')
        if laps_data is not None:
            print(f"Loaded {gp}/laps.csv with columns: {laps_data.columns.tolist()}")
            laps_data['Time'] = pd.to_timedelta(laps_data['Time'].astype(str).str.replace('0 days', '').str.strip())
            laps_data['Time'] = base_time + laps_data['Time']
            laps_data['GrandPrix'] = gp
            if 'Compound' in laps_data.columns:
                laps_data['Compound'] = laps_data['Compound'].replace(compound_map)
                # 제거할 Compound 목록
                drop_compounds = ['TEST', 'TEST_UNKNOWN', 'UNKNOWN']
                # 해당 Compound인 행 제거
                laps_data = laps_data[~laps_data['Compound'].isin(drop_compounds)].reset_index(drop=True)
            else:
                print(f"Warning: No 'Compound' column in {gp}/laps.csv, skipping compound processing")
            all_laps_data.append(laps_data)  # 항상 추가
        else:
            print(f"Warning: Failed to load {gp}/laps.csv")

        weather_data = azure_access.read_csv_by_data_frame(f'{gp}/weather_data.csv')
        if weather_data is not None:
            print(f"Loaded {gp}/weather_data.csv with columns: {weather_data.columns.tolist()}")
            weather_data['Time'] = pd.to_timedelta(weather_data['Time'].astype(str).str.replace('0 days', '').str.strip())
            weather_data['Time'] = base_time + weather_data['Time']
            weather_data['GrandPrix'] = gp
            weather_data = weather_data.sort_values('Time')
            all_weather_data.append(weather_data)
        else:
            print(f"Warning: Failed to load {gp}/weather_data.csv")

    # 데이터 결합
    laps_data = pd.concat(all_laps_data, ignore_index=True)

    # --- 이 코드를 데이터 로딩 및 결합 직후에 추가하세요 ---

    print(f"처리 전 데이터 행 수: {len(laps_data)}")
    print(f"처리 전 'Unknown' 데이터 수: {len(laps_data[laps_data['Compound'] == 'Unknown'])}")

    # 'Compound' 컬럼 값이 'Unknown'이 아닌 행들만 남깁니다.
    laps_data = laps_data[laps_data['Compound'] != 'Unknown'].reset_index(drop=True)

    print(f"처리 후 'Unknown' 데이터 수: {len(laps_data[laps_data['Compound'] == 'Unknown'])}")
    print(f"처리 후 데이터 행 수: {len(laps_data)}")

    weather_data = pd.concat(all_weather_data, ignore_index=True)

    if not weather_data.empty:
        print(f"Loaded weather.csv with columns: {weather_data.columns.tolist()}")

    print("laps_data shape:", laps_data.shape)
    print("laps_data columns:", laps_data.columns)
    print("Sample PitInTime and PitOutTime:\n", laps_data[['LapNumber', 'PitInTime', 'PitOutTime']].head(30))

    # 데이터 전처리 및 정렬 (수정된 부분)
    def fix_time_sorting(df, time_col='Time', group_col='GrandPrix'):
        """
        시간 컬럼의 정렬 문제를 수정하는 함수
        """
        # NaN 값 제거
        df = df.dropna(subset=[time_col]).copy()
        
        # datetime으로 변환
        df[time_col] = pd.to_datetime(df[time_col])
        
        # 각 그룹별로 정렬 확인 및 수정
        fixed_groups = []
        
        for group_name in df[group_col].unique():
            group_data = df[df[group_col] == group_name].copy()
            
            # 시간순으로 정렬
            group_data = group_data.sort_values(time_col).reset_index(drop=True)
            
            # 정렬 확인
            if not group_data[time_col].is_monotonic_increasing:
                print(f"Warning: {group_name} has non-monotonic time values, fixing...")
                # 중복된 시간값이 있을 경우 미세하게 조정
                duplicated = group_data[time_col].duplicated()
                if duplicated.any():
                    print(f"Found {duplicated.sum()} duplicate time values in {group_name}")
                    # 중복된 시간에 microsecond 추가
                    for i, is_dup in enumerate(duplicated):
                        if is_dup:
                            group_data.iloc[i, group_data.columns.get_loc(time_col)] += pd.Timedelta(microseconds=i)
            
            fixed_groups.append(group_data)
            print(f"Fixed {group_name}: {len(group_data)} rows, time range: {group_data[time_col].min()} to {group_data[time_col].max()}")
        
        return pd.concat(fixed_groups, ignore_index=True)

    # 시간 정렬 수정
    print("\n=== Fixing time sorting issues ===")
    laps_data = fix_time_sorting(laps_data)
    weather_data = fix_time_sorting(weather_data)

    # 병합 전 최종 정렬 및 확인
    print("\n=== Final sorting and verification ===")
    laps_data = laps_data.sort_values(['GrandPrix', 'Time']).reset_index(drop=True)
    weather_data = weather_data.sort_values(['GrandPrix', 'Time']).reset_index(drop=True)

    # 정렬 상태 최종 확인
    print("Final sorting verification:")
    for gp in laps_data['GrandPrix'].unique():
        laps_gp = laps_data[laps_data['GrandPrix'] == gp]['Time']
        weather_gp = weather_data[weather_data['GrandPrix'] == gp]['Time'] if gp in weather_data['GrandPrix'].unique() else pd.Series(dtype='datetime64[ns]')
        
        laps_sorted = laps_gp.is_monotonic_increasing
        weather_sorted = weather_gp.is_monotonic_increasing if not weather_gp.empty else True
        
        print(f"{gp}: laps_sorted={laps_sorted}, weather_sorted={weather_sorted}")
        
        if not laps_sorted or not weather_sorted:
            print(f"ERROR: {gp} is not properly sorted!")
            break

    # 시간 차이 계산 (수정)
    print("\n=== Analyzing time differences ===")
    laps_times = laps_data.sort_values(['GrandPrix', 'Time'])['Time']
    weather_times = weather_data.sort_values(['GrandPrix', 'Time'])['Time']
    time_diff = pd.Series(dtype='timedelta64[ns]', index=laps_times.index)
    for idx in laps_times.index:
        nearest_idx = weather_times.searchsorted(laps_times[idx], side='left') - 1
        if nearest_idx >= 0 and nearest_idx < len(weather_times):
            nearest_weather_time = weather_times.iloc[nearest_idx]
            time_diff[idx] = abs(laps_times[idx] - nearest_weather_time)
        else:
            time_diff[idx] = pd.Timedelta(seconds=0)  # 기본값 설정
    print(f"Max time difference: {time_diff.max()}")
    print(f"Mean time difference: {time_diff[time_diff > pd.Timedelta(0)].mean().total_seconds():.2f} seconds")
    print(f"Median time difference: {time_diff[time_diff > pd.Timedelta(0)].median().total_seconds():.2f} seconds")

    # Fixed merge section - replace the existing merge attempt section

    print("\n=== Attempting merge (FIXED) ===")

    # Check if we have weather data to merge
    if not weather_data.empty:
        # Create separate dataframes for each Grand Prix and merge individually
        merged_groups = []
        
        for gp in laps_data['GrandPrix'].unique():
            print(f"Processing {gp}...")
            
            # Get data for this specific Grand Prix
            laps_gp = laps_data[laps_data['GrandPrix'] == gp].copy()
            weather_gp = weather_data[weather_data['GrandPrix'] == gp].copy()
            
            if weather_gp.empty:
                print(f"  No weather data for {gp}, using default values")
                # Add default weather values
                if not weather_data.empty:
                    avg_track_temp = weather_data['TrackTemp'].mean()
                    avg_air_temp = weather_data['AirTemp'].mean() if 'AirTemp' in weather_data.columns else 25.0
                    laps_gp['TrackTemp'] = avg_track_temp
                    if 'AirTemp' in weather_data.columns:
                        laps_gp['AirTemp'] = avg_air_temp
                    if 'Humidity' in weather_data.columns:
                        laps_gp['Humidity'] = weather_data['Humidity'].mean()
                    if 'Pressure' in weather_data.columns:
                        laps_gp['Pressure'] = weather_data['Pressure'].mean()
                    if 'WindDirection' in weather_data.columns:
                        laps_gp['WindDirection'] = weather_data['WindDirection'].mean()
                    if 'WindSpeed' in weather_data.columns:
                        laps_gp['WindSpeed'] = weather_data['WindSpeed'].mean()
                    if 'Rainfall' in weather_data.columns:
                        laps_gp['Rainfall'] = 0
                merged_groups.append(laps_gp)
                continue
            
            # Sort by Time only (critical for merge_asof)
            laps_gp = laps_gp.sort_values('Time').reset_index(drop=True)
            weather_gp = weather_gp.sort_values('Time').reset_index(drop=True)
            
            # Verify sorting
            if not laps_gp['Time'].is_monotonic_increasing:
                print(f"  ERROR: laps data for {gp} is not properly time-sorted")
                continue
            if not weather_gp['Time'].is_monotonic_increasing:
                print(f"  ERROR: weather data for {gp} is not properly time-sorted")
                continue
            
            try:
                # Perform merge_asof for this Grand Prix
                merged_gp = pd.merge_asof(
                    laps_gp,
                    weather_gp.drop(columns=['GrandPrix']),  # Remove GrandPrix to avoid duplication
                    on='Time',
                    direction='nearest',
                    tolerance=pd.Timedelta(minutes=20)  # 20 minute tolerance
                )
                
                print(f"  Successfully merged {len(merged_gp)} rows")
                merged_groups.append(merged_gp)
                
            except Exception as e:
                print(f"  Merge failed for {gp}: {e}")
                # Add default weather values for this GP
                if not weather_data.empty:
                    avg_track_temp = weather_data['TrackTemp'].mean()
                    laps_gp['TrackTemp'] = avg_track_temp
                    if 'AirTemp' in weather_data.columns:
                        laps_gp['AirTemp'] = weather_data['AirTemp'].mean()
                merged_groups.append(laps_gp)
        
        # Combine all merged groups
        if merged_groups:
            laps_data = pd.concat(merged_groups, ignore_index=True)
            print("Merge completed successfully!")
            print(f"Final merged data shape: {laps_data.shape}")
            
            # Check what weather columns were successfully merged
            weather_cols = ['TrackTemp', 'AirTemp', 'Humidity', 'Pressure', 'WindDirection', 'WindSpeed', 'Rainfall']
            available_weather_cols = [col for col in weather_cols if col in laps_data.columns]
            print(f"Available weather columns: {available_weather_cols}")
            
            # Show weather data statistics
            for col in available_weather_cols:
                if laps_data[col].notna().sum() > 0:
                    print(f"  {col}: {laps_data[col].min():.1f} to {laps_data[col].max():.1f} (mean: {laps_data[col].mean():.1f})")
        else:
            print("No data groups were successfully processed")
            
    else:
        print("No weather data available for merging")
        # Add default weather columns
        laps_data['TrackTemp'] = 31.2  # Default track temperature
        laps_data['AirTemp'] = 25.0    # Default air temperature

    print(f"Final data shape after merge: {laps_data.shape}")
    print(f"Columns after merge: {laps_data.columns.tolist()}")

    # 나머지 전처리
    print("\n=== Data preprocessing ===")

    # Boolean 열 처리
    boolean_cols = ['IsPersonalBest', 'FreshTyre', 'Deleted', 'FastF1Generated', 'IsAccurate']
    if 'Rainfall' in laps_data.columns:
        boolean_cols.append('Rainfall')

    for col in boolean_cols:
        if col in laps_data.columns:
            laps_data[col] = pd.to_numeric(laps_data[col], errors='coerce').fillna(0).astype('Int64')

    # Special handling for pit-related columns
    laps_data['IsPitStop'] = laps_data['PitInTime'].notnull().astype('Int64')

    # 'LapTime' 및 섹터 타임 변환
    time_cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
    for col in time_cols:
        if col in laps_data.columns and laps_data[col].dtype != 'float64':
            laps_data[col] = laps_data[col].apply(lambda x: pd.to_timedelta(str(x).replace('0 days', '')).total_seconds() if pd.notna(x) else np.nan)

    # Drop rows where 'LapTime' is missing
    laps_data = laps_data.dropna(subset=['LapTime', 'Compound', 'TyreLife'])

    # SpeedI1, SpeedFL 보간
    if 'SpeedI1' in laps_data.columns:
        laps_data['SpeedI1'] = laps_data['SpeedI1'].interpolate(method='linear').fillna(laps_data['SpeedI1'].median())
    if 'SpeedFL' in laps_data.columns:
        laps_data['SpeedFL'] = laps_data['SpeedFL'].interpolate(method='linear').fillna(laps_data['SpeedFL'].median())

    # 피처 엔지니어링: 타이어 마모율
    laps_data['TyreWear'] = laps_data.groupby(['Driver', 'Stint'])['TyreLife'].transform('max') - laps_data['TyreLife']

    # 피처 엔지니어링: LapTime 변화율
    laps_data['LapTime_Delta'] = laps_data.groupby(['Driver', 'Stint'])['LapTime'].diff().fillna(0)
    if 'SpeedI1' in laps_data.columns:
        laps_data['SpeedI1_Delta'] = laps_data.groupby(['Driver', 'Stint'])['SpeedI1'].diff().fillna(0)
    laps_data['TyreWear_Rate'] = laps_data.groupby(['Driver', 'Stint'])['TyreWear'].diff().fillna(0) / laps_data['LapNumber']

    # Define column categories
    numerical_cols = ['LapNumber', 'Stint', 'TyreLife', 'Position', 'LapTime_Delta', 'TyreWear_Rate']
    if 'SpeedI1' in laps_data.columns:
        numerical_cols.extend(['SpeedI1', 'SpeedI1_Delta'])
    if 'SpeedFL' in laps_data.columns:
        numerical_cols.append('SpeedFL')
    if 'TrackTemp' in laps_data.columns:
        numerical_cols.append('TrackTemp')
    if 'AirTemp' in laps_data.columns:
        numerical_cols.append('AirTemp')
    if 'Rainfall' in laps_data.columns:
        numerical_cols.append('Rainfall')

    categorical_cols = ['Compound', 'Driver', 'Team']
    if 'TrackStatus' in laps_data.columns:
        categorical_cols.append('TrackStatus')

    # Boolean 열 처리 (존재하는 컬럼만)
    boolean_cols_candidate = ['IsPersonalBest', 'FreshTyre', 'Deleted', 'FastF1Generated', 'IsAccurate', 'Rainfall']
    boolean_cols = [col for col in boolean_cols_candidate if col in laps_data.columns]

    print(f"Processing boolean columns: {boolean_cols}")
    for col in boolean_cols:
        # NaN을 0으로 대체하고 nullable 정수형으로 변환
        laps_data[col] = pd.to_numeric(laps_data[col], errors='coerce').fillna(0).astype('Int64')

    # Special handling for pit-related columns
    laps_data['IsPitStop'] = laps_data['PitInTime'].notnull().astype('Int64')
    print("Created IsPitStop column")

    # 'LapTime' 및 섹터 타임 변환
    time_cols_candidate = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
    time_cols = [col for col in time_cols_candidate if col in laps_data.columns]

    print(f"Processing time columns: {time_cols}")
    for col in time_cols:
        if laps_data[col].dtype != 'float64':
            print(f"Converting {col} from {laps_data[col].dtype} to seconds")
            laps_data[col] = laps_data[col].apply(
                lambda x: pd.to_timedelta(str(x).replace('0 days', '')).total_seconds() 
                if pd.notna(x) else np.nan
            )

    # Drop rows where 'LapTime' is missing
    if 'LapTime' in laps_data.columns:
        before_drop = len(laps_data)
        laps_data = laps_data.dropna(subset=['LapTime'])
        after_drop = len(laps_data)
        print(f"Dropped {before_drop - after_drop} rows with missing LapTime")

    # SpeedI1, SpeedFL 보간 (존재하는 컬럼만)
    speed_cols = ['SpeedI1', 'SpeedFL', 'SpeedI2', 'SpeedST']
    for col in speed_cols:
        if col in laps_data.columns:
            print(f"Interpolating {col}")
            laps_data[col] = laps_data[col].interpolate(method='linear').fillna(laps_data[col].median())

    # 피처 엔지니어링: 타이어 마모율
    if all(col in laps_data.columns for col in ['Driver', 'Stint', 'TyreLife']):
        laps_data['TyreWear'] = laps_data.groupby(['Driver', 'Stint'])['TyreLife'].transform('max') - laps_data['TyreLife']
        print("Created TyreWear feature")

    # 피처 엔지니어링: LapTime 변화율
    if all(col in laps_data.columns for col in ['Driver', 'Stint', 'LapTime']):
        laps_data['LapTime_Delta'] = laps_data.groupby(['Driver', 'Stint'])['LapTime'].diff().fillna(0)
        print("Created LapTime_Delta feature")

    if all(col in laps_data.columns for col in ['Driver', 'Stint', 'SpeedI1']):
        laps_data['SpeedI1_Delta'] = laps_data.groupby(['Driver', 'Stint'])['SpeedI1'].diff().fillna(0)
        print("Created SpeedI1_Delta feature")

    if all(col in laps_data.columns for col in ['Driver', 'Stint', 'TyreWear', 'LapNumber']):
        laps_data['TyreWear_Rate'] = laps_data.groupby(['Driver', 'Stint'])['TyreWear'].diff().fillna(0) / laps_data['LapNumber']
        print("Created TyreWear_Rate feature")

    # Define column categories (존재하는 컬럼만)
    numerical_cols_candidate = [
        'LapNumber', 'Stint', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 
        'TyreLife', 'Position', 'TrackTemp', 'AirTemp', 'Humidity', 'Pressure', 'WindDirection', 'WindSpeed',
        'LapTime_Delta', 'SpeedI1_Delta', 'TyreWear_Rate', 'TyreWear', 'Rainfall'
    ]
    numerical_cols = [col for col in numerical_cols_candidate if col in laps_data.columns]

    categorical_cols_candidate = ['Compound', 'TrackStatus', 'Driver', 'Team', 'DriverNumber']
    categorical_cols = [col for col in categorical_cols_candidate if col in laps_data.columns]

    print(f"\nFinal column categories:")
    print(f"Numerical columns ({len(numerical_cols)}): {numerical_cols}")
    print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols}")

    # 데이터 품질 체크
    print(f"\nData quality check:")
    print(f"Total rows: {len(laps_data)}")
    print(f"Total columns: {len(laps_data.columns)}")

    # Missing values check for key columns
    key_columns = ['LapTime', 'Driver', 'LapNumber', 'GrandPrix', 'Compound', 'TyreLife']
    for col in key_columns:
        if col in laps_data.columns:
            missing_count = laps_data[col].isnull().sum()
            print(f"{col}: {missing_count} missing values ({missing_count/len(laps_data)*100:.2f}%)")

    # PitStop 관련 통계
    if 'IsPitStop' in laps_data.columns:
        pit_stops = laps_data[laps_data['IsPitStop'] == 1]
        print(f"\nPit stop statistics:")
        print(f"Total pit stops: {len(pit_stops)}")
        print(f"Pit stops with PitInTime: {pit_stops['PitInTime'].notna().sum()}")
        print(f"Pit stops with PitOutTime: {pit_stops['PitOutTime'].notna().sum()}")

    # GrandPrix별 데이터 분포
    if 'GrandPrix' in laps_data.columns:
        print(f"\nData distribution by GrandPrix:")
        gp_counts = laps_data['GrandPrix'].value_counts()
        for gp, count in gp_counts.items():
            print(f"  {gp}: {count} rows")


    print("\nData preprocessing completed successfully!")

    print("\nPHASE 2: 데이터셋 분리 시작")
    laps_data_grouped = laps_data.copy()
    laps_data_single = laps_data[laps_data['GrandPrix'].str.contains(track_name, na=False)].copy()

    dry_laps_data_grouped = laps_data_grouped[laps_data_grouped['Rainfall'] == 0].copy()
    wet_laps_data_grouped = laps_data_grouped[laps_data_grouped['Rainfall'] == 1].copy()
    dry_laps_data_single = laps_data_single[laps_data_single['Rainfall'] == 0].copy()

    has_wet_data = not wet_laps_data_grouped.empty
    print("PHASE 2: 완료")



    train_data = dry_laps_data_grouped.copy()

    # preprocessor 정의
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), numerical_cols),
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # handle_unknown 추가
            ]), categorical_cols)
        ],
        remainder='drop'
    )

    # 타겟 변수 설정
    X = train_data[numerical_cols + categorical_cols]
    y_laptime = train_data['LapTime'].values
    y_pitstop = (train_data['PitInTime'].notna()).astype(int).values
    y_tyrewear = train_data['TyreWear'].values
    y_multi = np.column_stack((y_laptime, y_pitstop, y_tyrewear))


    # --- 3. 데이터 분할 ---
    X_train, X_test, y_multi_train, y_multi_test = train_test_split(
        X, y_multi, test_size=0.2, random_state=42
    )
    y_laptime_train, y_pitstop_train, y_tyrewear_train = y_multi_train.T
    y_laptime_test, y_pitstop_test, y_tyrewear_test = y_multi_test.T


    # ==============================================================================
    # ⭐️ 찾은 최적 파라미터로 최종 모델 학습 (수정된 최종 버전) ⭐️
    # ==============================================================================

    # preprocessor 학습
    preprocessor.fit(X_train)

    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    # 5. 오버샘플링 (ADASYN) - 변환된 데이터를 사용
    print(">>> 피트스톱 데이터 오버샘플링 (ADASYN)...")
    adasyn = ADASYN(sampling_strategy=0.3, random_state=42)
    # X_train이 아닌 X_train_transformed를 사용해야 합니다.
    X_train_pit_res, y_pitstop_train_res = adasyn.fit_resample(X_train_transformed, y_pitstop_train)
    print(">>> 오버샘플링 완료.")




    # Optuna 최적화 실행
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=30, n_jobs=-1)

    # 최적 파라미터 출력
    print("Best parameters:", study.best_params)
    print("Best score:", study.best_value)

    # 최적 모델 학습 및 평가 (예시)
    best_params = study.best_params


    # --- 1. 다중 출력 회귀 모델 (LapTime, TyreWear 예측) ---
    multi_output_params = best_params.copy()
    # 회귀 모델에 불필요한 scale_pos_weight 파라미터 제거
    multi_output_params.pop('scale_pos_weight', None) 
    # 회귀 모델의 objective 명시
    multi_output_params['objective'] = 'reg:squarederror'

    multi_output_model = xgb.XGBRegressor(**multi_output_params)
    # X_train_transformed와 y_multi_train[:, [0, 2]]로 학습
    multi_output_model.fit(X_train_transformed, y_multi_train[:, [0, 2]])
    print("Final Multi-Output Regressor trained with best parameters.")


    # --- 2. 피트스톱 분류 모델 ---
    pitstop_params = best_params.copy()
    # 분류 모델의 objective 명시 (기존 값을 덮어씀)
    pitstop_params['objective'] = 'binary:logistic'

    xgb_pitstop = xgb.XGBClassifier(**pitstop_params)
    # X_train_pit_res, y_pitstop_train_res (오버샘플링된 데이터)로 학습
    xgb_pitstop.fit(X_train_pit_res, y_pitstop_train_res)
    print("Final PitStop Classifier trained with best parameters.")


    # PHASE 3: Dry Strategy 파라미터 계산
    print("\nPHASE 3: Dry Strategy 파라미터 계산 시작")
    # ML 모델 관련 파라미터 (Grouped 데이터 사용)
    print(">>> Grouped 데이터로 ML 기반 파라미터 계산...")
    X_dry_test_grouped = preprocessor.transform(dry_laps_data_grouped)
    y_dry_pitstop_test_grouped = (dry_laps_data_grouped['PitInTime'].notna()).astype(int)
    y_dry_pitstop_proba_grouped = xgb_pitstop.predict_proba(X_dry_test_grouped)[:, 1]
    dry_pit_proba_threshold = calculate_pit_proba_threshold(y_dry_pitstop_test_grouped, y_dry_pitstop_proba_grouped)
    dry_tyrelife_threshold = calculate_tyre_wear_threshold(dry_laps_data_grouped)

    # 컴파운드 성능 파라미터 (Grouped 데이터 사용)
    dry_compound_performance = calculate_all_compound_performance(dry_laps_data_grouped)

    # 트랙 고유 파라미터 (Single 데이터 사용)
    print("\n>>> Single 데이터로 트랙 고유 파라미터 계산...")
    dry_compound_choice_thresholds = find_compound_choice_thresholds(dry_laps_data_single, track_name)
    dry_pit_stop_time = calculate_pit_stop_time(dry_laps_data_single, track_name)
    print("PHASE 3: 완료")


    # PHASE 4: Wet Strategy 파라미터 계산
    print("\nPHASE 4: Wet Strategy 파라미터 계산 시작")
    wet_compound_performance = {}
    wet_pit_stop_time = 24.5  # 기본값 또는 계산값
    if has_wet_data:
        # Wet 데이터는 양이 적으므로 항상 Grouped 데이터를 사용
        wet_compound_performance = calculate_all_compound_performance(wet_laps_data_grouped)
        wet_pit_stop_time = calculate_pit_stop_time(wet_laps_data_grouped, track_name)
        print("PHASE 4: 완료")
    else:
        print("Wet 데이터가 없어 PHASE 4를 건너뜁니다.")



    # ==============================================================================
    # PHASE 5: 결과 통합 및 최종 JSON 파일 저장
    # ==============================================================================
    print("\nPHASE 5: 최종 JSON 파일 생성 시작")

    final_json_data = {
        "metadata": {
            "track": track_name,
            "version": "2.0",
            "description": f"{track_name} GP F1 strategy simulation parameters with Dry/Wet separation.",
            "created_at": datetime.now().isoformat()
        },
        "dry_strategy_params": {
            "description": "Parameters for when Rainfall is 0. Uses ML-based predictions.",
            "ml_hyper_parameters": best_params, # Optuna에서 찾은 최적 파라미터
            "pit_proba_threshold": dry_pit_proba_threshold,
            "TyreLife_threshold": dry_tyrelife_threshold,
            # "compound_choice_thresholds": dry_pit_stop_data_result,
            "pit_stop_time": dry_pit_stop_time,
            "compound_performance": {
                # dry_compound_performance에서 SOFT, MEDIUM, HARD 키가 있는 항목만 필터링
                compound: dry_compound_performance[compound]
                for compound in ['SOFT', 'MEDIUM', 'HARD']
                if compound in dry_compound_performance
            }
        }
    }

    if has_wet_data:
        final_json_data["wet_strategy_params"] = {
            "description": "Parameters for when Rainfall is 1. Uses rule-based logic.",
            "strategy_rule": "When rainfall begins, recommend immediate pit for INTERMEDIATE tires.",
            "pit_stop_time": wet_pit_stop_time,
            "compound_performance": {
                # wet_compound_performance에서 INTERMEDIATE, WET 키가 있는 항목만 필터링
                compound: wet_compound_performance[compound]
                for compound in ['INTERMEDIATE', 'WET']
                if compound in wet_compound_performance
            }
        }

    # --- 이 부분이 핵심적인 수정 사항입니다 ---
    print(">>> NumPy 타입을 Python 기본 타입으로 변환 중...")
    final_json_data_serializable = convert_numpy_types(final_json_data)
    print(">>> 변환 완료.")
    # -----------------------------------------

    # 최종 JSON 파일 저장
    output_filename = f"/home/azureuser/cloudfiles/code/Users/project/src/tyre_strategy_module/json_result/complete/{track_name}_complete_strategy_params_v2.json"
    with open(output_filename, 'w', encoding='utf-8') as f:
        # 변환된 객체를 json.dump에 전달합니다.
        json.dump(final_json_data_serializable, f, indent=2, ensure_ascii=False)

    print(f"\n✅ 최종 파라미터 파일이 '{output_filename}'으로 성공적으로 저장되었습니다.")
    print("PHASE 5: 완료")



    # 해당 트랙의 파라미터 파일 로드
    params_filename = f"/home/azureuser/cloudfiles/code/Users/project/src/tyre_strategy_module/json_result/complete/{track_name}_complete_strategy_params_v2.json"
    try:
        with open(params_filename, 'r', encoding='utf-8') as f:
            params_data = json.load(f)
        print(f"✅ '{params_filename}' 파라미터 로딩 완료.")
    except FileNotFoundError:
        print(f"❌ 에러: 파라미터 파일 '{params_filename}'을 찾을 수 없습니다.")
        exit()

    best_params = params_data["dry_strategy_params"]["ml_hyper_parameters"]

    # 시뮬레이션용 랩 데이터 준비 (이 부분은 모델 예측이 끝난 후 실행되어야 함)
    # (X_test, multi_output_model, xgb_pitstop 등이 이미 메모리에 있다고 가정)
    print(">>> 시뮬레이션용 랩 데이터 준비 중...")
    y_pred_multi = multi_output_model.predict(X_test_transformed)
    y_pred_laptime = y_pred_multi[:, 0]
    y_pred_tyrewear = y_pred_multi[:, 1]
    y_pred_pitstop_proba = xgb_pitstop.predict_proba(X_test_transformed)[:, 1]
    # ... (이전과 동일한 lap_data_df 생성 로직) ...
    # 시뮬레이션 입력 데이터프레임 생성
    lap_numbers = X_test['LapNumber'].values
    rainfall = X_test['Rainfall'].values if 'Rainfall' in X_test.columns else np.zeros(len(X_test))
    track_temp = X_test['TrackTemp'].values if 'TrackTemp' in X_test.columns else np.full(len(X_test), 30.0)
    lap_time_delta = X_test['LapTime_Delta'].values if 'LapTime_Delta' in X_test.columns else np.zeros(len(X_test))

    lap_data_df = pd.DataFrame({
        'lap': lap_numbers,
        'laptime': y_pred_laptime,
        'tyre_wear': y_pred_tyrewear,
        'pit_proba': y_pred_pitstop_proba,
        'lap_delta': lap_time_delta,
        'rainfall': rainfall,
        'track_temp': track_temp
    })
    lap_data_df = lap_data_df.groupby('lap').mean().reset_index().sort_values('lap')
    print(f"✅ 랩 데이터 준비 완료. (총 {len(lap_data_df)} 랩)")

    
    for initial_compound in compound :

        # ------------------------------------------------------------------------------
        # [ STEP 4: 유전 알고리즘을 이용한 최적 전략 탐색 ]
        # ------------------------------------------------------------------------------
        print("\n>>> STEP 4: 유전 알고리즘으로 최적 전략 탐색 시작...")

        # ==============================================================================
        #           <<<< 유전 알고리즘 기반 F1 전략 탐색 엔진 (수정 완료) >>>>
        # ==============================================================================

        # --- 1. 유전 알고리즘 기본 설정 (RuntimeWarning 방지 코드 추가) ---
        # 이전에 생성된 클래스가 있다면 삭제하여 경고 메시지 방지
        if hasattr(creator, "FitnessMin"):
            del creator.FitnessMin
        if hasattr(creator, "Individual"):
            del creator.Individual

        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMin)

        toolbox = base.Toolbox()


        # --- 2. 유전자(Gene) 및 염색체(Chromosome) 정의 (수정된 버전) ---

        COMPOUNDS = ['SOFT', 'MEDIUM', 'HARD']

        # toolbox에 위 함수를 'individual'이라는 이름으로 등록 (이 부분은 동일)
        toolbox.register("individual", create_random_strategy, race_laps=race_laps)
        # toolbox에 'population'을 individual의 리스트로 정의 (이 부분은 동일)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)

        # --- 4. 교배, 돌연변이, 선택 연산자 정의 (규칙 준수 버전) ---
        # (custom_mutate, custom_mate 등 이전과 동일)
        toolbox.register("evaluate", evaluate_hybrid_strategy) # <--- 업그레이드된 함수로 교체!
        toolbox.register("mate", custom_mate)
        toolbox.register("mutate", custom_mutate, low=[10,0,0,0], up=[race_laps-10, 2, race_laps-10, 2], indpb=0.2)
        toolbox.register("select", tools.selTournament, tournsize=3)

        # --- 5. 유전 알고리즘 실행 (이전과 동일) ---
        print("\n" + "="*60)
        print(f"GENETIC ALGORITHM: SEARCHING FOR OPTIMAL STRATEGY (v2, Rule-Compliant)...")
        print("="*60)
        population = toolbox.population(n=50)
        NGEN = 40
        CXPB, MUTPB = 0.5, 0.2
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean); stats.register("min", np.min); stats.register("max", np.max)
        result_pop, logbook = algorithms.eaSimple(population, toolbox, CXPB, MUTPB, NGEN, stats=stats, verbose=True)

        # --- 6. 최종 결과 출력 (이전과 동일) ---
        best_individual = tools.selBest(result_pop, k=1)[0]
        best_fitness = best_individual.fitness.values[0]
        pit1_lap, pit1_tyre_idx, pit2_lap, pit2_tyre_idx = best_individual


        # ------------------------------------------------------------------------------
        # [ STEP 5: 최종 결과 출력 ]
        # ------------------------------------------------------------------------------
        # (GA 실행 후 best_individual을 해석하여 결과를 출력하는 코드)
        # ...

        print("\n" + "="*60)
        print("OPTIMAL STRATEGY (FOUND BY GENETIC ALGORITHM)")
        print("="*60)

        pit_stops = []

        if pit1_lap > 0:
            pit_stops.append({'lap': int(pit1_lap), 'tyre': COMPOUNDS[pit1_tyre_idx]})
        if pit2_lap > 0:
            pit_stops.append({'lap': int(pit2_lap), 'tyre': COMPOUNDS[pit2_tyre_idx]})

        # 만약을 위해 피트스톱 랩 순서대로 정렬
        pit_stops = sorted(pit_stops, key=lambda x: x['lap'])

        # --- 2. 최종 결과 요약 출력 ---


        # 한 줄 요약 문자열 생성
        strategy_summary = f"Start with {initial_compound} -> "
        if not pit_stops:
            strategy_summary = f"0-Stop: Run entire race on {initial_compound}"
        else:
            for pit in pit_stops:
                strategy_summary += f"Lap {pit['lap']} PIT to {pit['tyre']} -> "
        strategy_summary += "Finish"

        print(f"🏆 Best Strategy Found: {strategy_summary}")
        print(f"⏱️ Estimated Race Time: {best_fitness/60:.2f} minutes")
        print(f"🔧 Total Pit Stops: {len(pit_stops)}")

        # --- 3. 스틴트별 상세 전략 출력 ---
        if not pit_stops:
            print(f"\n📅 STINT DETAILS:")
            print(f" └─ Stint 1: Laps 1-{race_laps} ({race_laps} Laps) on {initial_compound}")
        else:
            print(f"\n📅 PIT STOP SCHEDULE:")
            current_compound = initial_compound
            stint_start_lap = 1
            
            for i, pit in enumerate(pit_stops):
                end_lap = pit['lap']
                stint_length = end_lap - stint_start_lap + 1
                print(f" ├─ Stint {i+1}: Laps {stint_start_lap}-{end_lap} ({stint_length} Laps) on {current_compound}")
                
                # 다음 스틴트를 위해 변수 업데이트
                current_compound = pit['tyre']
                stint_start_lap = end_lap + 1
            
            # 마지막 스틴트 출력
            final_stint_length = race_laps - stint_start_lap + 1
            if final_stint_length > 0:
                print(f" └─ Stint {len(pit_stops)+1}: Laps {stint_start_lap}-{race_laps} ({final_stint_length} Laps) on {current_compound}")


        # --- 3. ⭐️ 최종 결과를 JSON 형식으로 생성 및 출력 (API 출력용) ⭐️ ---
        final_result_json = {}
        final_result_json['track_name'] = track_name
        final_result_json['total_laps'] = race_laps
        final_result_json['estimated_race_time_seconds'] = best_fitness
        final_result_json['estimated_race_time_minutes'] = round(best_fitness / 60, 2)
        final_result_json['total_pit_stops'] = len(pit_stops)
        final_result_json['strategy_summary'] = strategy_summary

        # 스틴트 정보를 JSON 구조에 맞게 재구성
        stints_list = []
        current_compound = initial_compound
        stint_start_lap = 1

        if not pit_stops:
            stints_list.append({
                "stint_number": 1, "compound": current_compound, "start_lap": stint_start_lap,
                "end_lap": race_laps, "stint_length": race_laps
            })
        else:
            for i, pit in enumerate(pit_stops):
                end_lap = pit['lap']
                stint_length = end_lap - stint_start_lap + 1
                stints_list.append({
                    "stint_number": i + 1, "compound": current_compound, "start_lap": stint_start_lap,
                    "end_lap": end_lap, "stint_length": stint_length
                })
                current_compound = pit['tyre']
                stint_start_lap = end_lap + 1
            
            final_stint_length = race_laps - stint_start_lap + 1
            if final_stint_length > 0:
                stints_list.append({
                    "stint_number": len(pit_stops) + 1, "compound": current_compound, "start_lap": stint_start_lap,
                    "end_lap": race_laps, "stint_length": final_stint_length
                })

        final_result_json['stints'] = stints_list

        # JSON 형식으로 콘솔에 예쁘게 출력
        print("\n" + "="*60)
        print("FINAL STRATEGY AS JSON (FOR API OUTPUT)")
        print("="*60)
        # NumPy 타입을 안전하게 변환 후 출력
        final_result_json_serializable = convert_numpy_types(final_result_json)
        print(json.dumps(final_result_json_serializable, indent=2, ensure_ascii=False))

        # 최종 JSON 파일 저장
        output_filename = f"/home/azureuser/cloudfiles/code/Users/project/src/tyre_strategy_module/json_result/result/{track_name}_{initial_compound}_strategy_result.json"
        with open(output_filename, 'w', encoding='utf-8') as f:
            # 변환된 객체를 json.dump에 전달합니다.
            json.dump(final_result_json_serializable, f, indent=2, ensure_ascii=False)